In [74]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy
import sklearn
from math import *
from prettytable import PrettyTable
from functools import partial
from decimal import Decimal
import sympy
import time

plt.rcParams["font.sans-serif"] = ["Microsoft YaHei"]
plt.rcParams["axes.unicode_minus"] = False  # 解决无法显示符号的问题
# palette = "deep"
# sns.set(font="Microsoft YaHei", font_scale=1.2, palette=palette)  # 解决Seaborn中文显示问题

In [75]:
from mcm_cn2023.utils.util import *

In [76]:
'''
200,0,-197.2
0,0,-24.4
200,250,-65.20


0,250,-84.4
'''

'\n200,0,-197.2\n0,0,-24.4\n200,250,-65.20\n\n\n0,250,-84.4\n'

In [77]:
A = sympy.Point3D(nm2m(200 * 0.02), 0, 197.2)
B = sympy.Point3D(0, 0, 24.4)
C = sympy.Point3D(nm2m(200 * 0.02), nm2m(250 * 0.02), 65.20)
# 假设 C 抬高搞和 B 一样的高度
BC平均z = (B.z + C.z) / 2
C = sympy.Point3D(nm2m(200 * 0.02), nm2m(250 * 0.02), BC平均z)
B = sympy.Point3D(0, 0, BC平均z)

D = sympy.Point3D(0, nm2m(250 * 0.02), 84.4)

ABC = sympy.Plane(A, B, C)
BCD = sympy.Plane(B, C, D)

xoy = sympy.Plane((1, 0, 0), (0, 1, 0), (1, 1, 0))

In [78]:
ABC坡度 = 计算两平面夹角(ABC, xoy, False)
BCD坡度 = 计算两向量夹角(
    np.array(BCD.normal_vector).astype(float),
    -np.array(xoy.normal_vector).astype(float),
    False,
)

print(f"ABC坡度 {degrees(ABC坡度)}")
print(f"BCD坡度 {degrees(BCD坡度)}")

ABC坡度 1.5091350335178098
BCD坡度 0.39222203141528517


In [79]:
def 求解(坡度: float, 海水总深度: float, 斜坡地面长度: float, 长度计算函数):
    theta = radians(120)
    alpha = 坡度
    O = sympy.Point2D(0, 0)
    A1 = O
    C = sympy.Point2D(斜坡地面长度, tan(alpha) * 斜坡地面长度)
    斜坡 = sympy.Line2D(O, C)

    def aim(参考点A: sympy.Point2D):
        # print("======================")
        A = 参考点A
        A到M的水平距离 = (海水总深度 - A.y) * tan(theta / 2)
        M = sympy.Point2D(
            A.x + A到M的水平距离,
            海水总深度,
        )

        右侧扫描线 = sympy.Line2D(
            M,
            (M.x + A到M的水平距离, A.y),
        )
        # B = 斜坡.intersection(右侧扫描线)[0]
        Bpos = find_intersection(斜坡.p1, 斜坡.p2, 右侧扫描线.p1, 右侧扫描线.p2)
        B = sympy.Point2D(Bpos[0], Bpos[1])

        AB = sqrt((A.x - B.x) ** 2 + (A.y - B.y) ** 2)
        ANext = B - 斜坡.direction.unit * AB * 0.1
        return M, B, ANext, AB

    def solve():
        ArrM = []

        ANext = A1
        beforeBx = 0
        总长度 = 0
        while True:
            M, B, ANext, AB = aim(ANext)
            ArrM.append(M)
            print(f"B.x >= C.x {float(B.x)} {float(C.x)} 总长度：{总长度}")
            if B.x >= C.x or B.x - beforeBx < 1:
                break

            总长度 += 长度计算函数(float(M.x))
            beforeBx = float(B.x)

        return ArrM

    return solve()

In [80]:
求解1 = 求解(
    坡度=ABC坡度,
    海水总深度=abs(A.z),
    斜坡地面长度=nm2m(20 / sqrt(41)),
    长度计算函数=lambda x: 41 * x / 20,
)

B.x >= C.x 653.3092346362032 5784.67614035397 总长度：0
B.x >= C.x 1189.9684364852494 5784.67614035397 总长度：700.198859467793
B.x >= C.x 1684.4716870801956 5784.67614035397 总长度：2550.7508477011497
B.x >= C.x 2140.130444083586 5784.67614035397 总长度：5461.292787745031
B.x >= C.x 2559.9960416991294 5784.67614035397 总长度：9348.559761035853
B.x >= C.x 2946.880124033905 5784.67614035397 总长度：14135.827521071627
B.x >= C.x 3303.3734733677384 5784.67614035397 总长度：19752.398706912798
B.x >= C.x 3631.863359413982 5784.67614035397 总长度：26133.129415924966
B.x >= C.x 3934.5495257514917 5784.67614035397 总长度：33217.99296544219
B.x >= C.x 4213.458920481523 5784.67614035397 总长度：40951.67792206602
B.x >= C.x 4470.459269753955 5784.67614035397 总长度：49283.21770678996
B.x >= C.x 4707.271585058304 5784.67614035397 总长度：58165.64929558763
B.x >= C.x 4925.481688035088 5784.67614035397 总长度：67555.69872994159
B.x >= C.x 5126.550829983804 5784.67614035397 总长度：77413.49133132357
B.x >= C.x 5311.8254771815045 5784.67614035397 总长度：87702

In [82]:
求解2 = 求解(
    坡度=BCD坡度,
    海水总深度=abs(D.z),
    斜坡地面长度=nm2m(20 / sqrt(41)),
    长度计算函数=lambda x: 41 * x / 20,
)

B.x >= C.x 288.9441501949572 5784.67614035397 总长度：0
B.x >= C.x 542.8993021463331 5784.67614035397 总长度：299.67943072556665
B.x >= C.x 791.4978797988042 5784.67614035397 总长度：1126.1398003645065
B.x >= C.x 1034.852867228495 5784.67614035397 总长度：2468.2699295733237
B.x >= C.x 1273.0748653836574 5784.67614035397 总长度：4315.1930026638565
B.x >= C.x 1506.272142351055 5784.67614035397 总长度：6656.261624264566
B.x >= C.x 1734.5506825620691 5784.67614035397 总长度：9481.052980249688
B.x >= C.x 1958.0142349609055 5784.67614035397 总长度：12779.364100736984
B.x >= C.x 2176.7643601568443 5784.67614035397 总长度：16541.207223001184
B.x >= C.x 2390.900476581935 5784.67614035397 总长度：20756.80525219556
B.x >= C.x 2600.519905675057 5784.67614035397 总长度：25416.58731781882
B.x >= C.x 2805.7179161129316 5784.67614035397 总长度：30511.184423907565
B.x >= C.x 3006.5877671082753 5784.67614035397 总长度：36031.42519097751
B.x >= C.x 3203.220750794578 5784.67614035397 总长度：41968.33168777826
B.x >= C.x 3395.7062337169177 5784.67614035397 总长度：

In [83]:
跨越整个海域的额外长度 = nm2m(40 / sqrt(41))
跨越整个海域的额外长度

11569.352280707939